In [ ]:
TRACK_PATH = r'\\Desktop-0331tte\d\chiletv-testdata\track\none@30FPS'
TRACK_CSV = r'..\data\testdata_tracks.csv'
TRACK_LENGTH = 5
TRACK_FRAME_RATE = 30.0
TRACK_SAMPLES = 5000

# DATA_SHAPE:: 5000, TRACK_LENGTH * TRACK_FRAME_RATE , 4

In [ ]:
%matplotlib inline

import json
from pathlib import Path
from datetime import timedelta
from typing import List

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [ ]:
for file in Path(TRACK_PATH).glob('**/*.tracks.json'):
    with file.open('r', encoding='utf8') as fp:
        obj = json.load(fp)

### Methodology
It is necesary to obtain the bounding box of the faces present for every frame on all videos. The GPU-backed face detector allows to efficiently process many frames in parallel. Despite this, running the face detector for all frames adds up to a considerable time. Addtionally, the face detector is agnostic to previous results leading to undesirable gaps and jittering. A gap is a short secuence of frames where the face remains undetected before suddenly returning, while jittering presents itself when succesive high-frequency detections result in bounding box coordinates that sharply oscillate.

The number of frames to be processed by the face detector and their dimensions affect the duration of a detection operation. An intuitive solution is to reduce both of them as reducing the amount of data being transfered between GPU and principal memories the operation can be sped-up. The *speedup* factor stands for the relative decrease in time spent detecting and its given by the ratio between the video length and the time spent processing it.

Lowering the detection frame rate can mitigate the jittering while introducing regular periodic gaps on the bounding box sequence. These gaps can be filled with values that soften the transition between detected points. An interpolation estrategy can be selected to fill missing data while keeping this transition as smooth as possible. The tried estrategies correspond the implementations of spline interpolators for zeroth, first, second and third order, that are already available on the *Python* library *SciPy*.

To maintain a reference based on actual data, the high frequency samples require an smoothing operation. This low-pass filter takes the form of a moving window averaging past detections. The window sequence contributions can be adjusted by the relative temporal distance to the sampled item. Three smoothing operations were tested: Uniform weights, Inversely linear weights and Gaussian weights. From these the one with the lowest MSE was chosen. The smoothed sequence was used as reference as it is the closest to an ideal result.

> The weights for a window of size $n$ on a sequence of length $m$ are computed as follows for Uniform weights ($W^U$), Inversely linear weights ($W^L$) and Gaussian weights ($W^G$).
> 
> $\forall i \in [0, m]$ $\forall j \in [0, n]$
> 
> $W^U_{ij}=1$
> 
> $W^L_{ij}=\frac{1}{|t_{i-j}-t_i| + 1}$
> 
> $W^G_{ij}=\frac{1}{P \sqrt{2 \pi}} e^{-\frac{1}{2}(\frac{t_{i-j}-t_i}{P})^2}$
> 
> The weights are normalized by the total sum of the weights. Notice that $W^G$ corresponds to the Gaussian distribution which mean is at instant $t_i$ and a deviation of $P=\frac{1}{f_r}$ with $fr$ being the frame rate.

To find the speedup factors a random sample of 150 videos was selected to be run through the face detector with a frame rate of 30, 15, 6, and 3 FPS. To acomodate batches of 1024 frames, all videos were resized by a factor of 8 due to harware restraints. The found bounding boxes for the 30 FPS videos were selected to further analyse the interpolation and smoothing experiments as this frame rate is the maximum allowed value by the the acquired videos. From these results 5000 random samples of 5 seconds were extracted from a total of 3846 tracks.

## Questions
> + Does lowering the frame rate make an improovement on detection time?
    + By how much?
    + Thresholding the frame size to a maximum value may help too, what dimensions to use as threshold?
> + Averaging past detections decrease the jittering?
    + Which performs the best?
> + Which interpolation strategy is closer to the true detection?
    + Which is closer to the softened detection?

In [ ]:
TRACK_POINTS_FILE = r'../data/video_points.pkl'
REFERENCE_FRAME_RATE = 30
SAMPLE_LENGTH = 5.0
SAMPLE_NUM = 5000

In [ ]:
%matplotlib inline
import pickle
from pathlib import Path
from datetime import timedelta

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [ ]:
# Load track data
point_data_src = Path(TRACK_POINTS_FILE)
with point_data_src.open('rb') as fp:
    data_time, data_bbox, data_tlen = pickle.load(fp)
print('Number of tracks:', len(data_time))

In [ ]:
# Sampling tracks
np.random.seed(0)

# Pick random track and a random position to start a slice
track_samples = (np.random.random(SAMPLE_NUM) * len(data_time)).astype(int)
track_start = np.random.random(SAMPLE_NUM) * (np.float32(data_tlen)[track_samples] - SAMPLE_LENGTH)

test_time = []
test_bbox = []

for ti, ts in zip(track_samples, track_start):
    t_time = []
    t_bbox = []
    # Add the frames that lie between the picked slice bounds
    for i in range(len(data_time[ti])):
        if ts <= data_time[ti][i] < ts + SAMPLE_LENGTH:
            t_time.append(data_time[ti][i])
            t_bbox.append(data_bbox[ti][i])
    # Make slices start from 0
    test_time.append(np.float32(t_time) - t_time[0])
    test_bbox.append(np.float32(t_bbox))

# Results

In [ ]:
def gaussian(x, mu, sigma):
    return (1.0 / np.sqrt(2 * np.pi * sigma**2)) * np.exp(-((x - mu)**2) / (2 * sigma**2))

def soften(x, y, win_size, fn):
    soft_y = []
    for c in range(len(x)):
        istart = max(c - win_size, 0)
        iend = c + 1
        # iend = min(c + win_size, len(x)-1)
        weights = fn(x[istart:iend])
        weights = weights / np.sum(weights)
        soft_y.append(np.average(y[istart:iend], weights=weights, axis=0))
    return np.float32(soft_y)

def mse(x_t, x_p, axis=None):
    t_area = np.mean(x_t, axis=0)
    t_area = (t_area[1, 0] - t_area[0, 0]) * (t_area[1, 1] - t_area[0, 1])
    return np.mean((x_t - x_p)**2) / t_area

def get_center_area(bbox):
    cter = np.sqrt(np.sum((0.5 * np.sum(bbox, axis=1))**2, axis=-1))
    area = (bbox[:, 1, 0] - bbox[:, 0, 0]) * (bbox[:, 1, 1] - bbox[:, 0, 1])
    cter = cter - np.mean(cter)
    area = area - np.mean(area)
    return cter, area

def get_error(ldiff):
    udiff = -ldiff
    ldiff[ldiff < 0] = 0
    udiff[udiff < 0] = 0
    return (ldiff, udiff)

def select_times(in_times, in_period, out_period):
    out_times = np.round(in_times / in_period).astype(int)
    k = int(np.round(out_period / in_period))
    out_times = np.mod(out_times, k)
    return np.argwhere(out_times == 0)

In [ ]:
# Find the softnening strategy with the lowest error.
test_cases = {
    'Uniform': lambda x: np.ones_like(x),
    'Linear': lambda x: 1.0 / (np.abs(x - x[-1]) + 1.0),
    'Gaussian': lambda x: gaussian(x, x[-1], 1.0 / REFERENCE_FRAME_RATE),
}

soft_bbox = {k: [] for k in test_cases}
mse_errors = {k: [] for k in test_cases}

for label, op in test_cases.items():
    for true_time, true_bbox in zip(test_time, test_bbox): 
        soft_bbox[label].append(soften(true_time, true_bbox, 5, op))
        
        mse_err = mse(true_bbox, soft_bbox[label][-1])
        mse_errors[label].append(mse_err)

In [ ]:
table = ((label, mse_err) for label, mse_errs in mse_errors.items() for mse_err in mse_errs)
mse_df = pd.DataFrame(table, columns=['weight_type', 'mse'])
mse_df.head()

In [ ]:
mse_df.groupby('weight_type').agg(np.sum).plot.bar()

In [ ]:
example_idx = {
    'Min Error': np.argmin(mse_errors['Gaussian']),
    'Med Error': np.argsort(mse_errors['Gaussian'])[len(mse_errors['Gaussian'])//2],
    'Max Error': np.argmax(mse_errors['Gaussian']),
}

fig, axs = plt.subplots(nrows=len(example_idx),
                        ncols=2,
                        figsize=(4*len(example_idx), 12),
                        constrained_layout=True,
                        squeeze=False)

line_labels = {}
for i, (name, eidx) in enumerate(example_idx.items()):
    cter_ax = axs[i, 0]
    area_ax = axs[i, 1]
    
    true_time = test_time[eidx]
    true_bbox = test_bbox[eidx]

    true_cter, true_area = get_center_area(true_bbox)
    l = cter_ax.scatter(true_time, true_cter, marker='o', label='Data')
    l = area_ax.scatter(true_time, true_area, marker='o', label='Data')
    line_labels['Data'] = l

    for (label, op), m in zip(test_cases.items(), 'x*+'):
        soft_cter, soft_area = get_center_area(soft_bbox[label][eidx])
        mse_err = mse_errors[label][eidx]
        
        print(name + '\t' + label, mse_err)
        
        l = cter_ax.scatter(true_time, soft_cter, marker=m)
        l = area_ax.scatter(true_time, soft_area, marker=m)
        line_labels[label] = l

    cter_ax.set_title(name)
    area_ax.set_title(name)

labels, lines = zip(*line_labels.items())

fig.set_constrained_layout(False)
plt.tight_layout()

fig.suptitle('Effect of smoothing on BBox Center and Area', fontsize=16)
fig.legend(lines, labels, loc=('lower center'), ncol=len(labels))    
fig.subplots_adjust(bottom=0.05, top=0.925)
plt.show()

In [ ]:
interp_type = ['zero', 'slinear', 'quadratic', 'cubic']
frame_rates = [15, 6, 3]
soft_kind = 'Gaussian'

table = []
for true_time, true_bbox, sref_bbox in zip(test_time, test_bbox, soft_bbox[soft_kind]):
    for interp_kind in interp_type:
        for frame_rate in frame_rates:
            idx = select_times(true_time, 1.0 / REFERENCE_FRAME_RATE, 1.0 / frame_rate)
            
            # Create the interpolated model from the subsampled data
            sub_time = true_time[idx].flatten()
            sub_bbox = true_bbox[idx].reshape(-1, 2, 2)
            
            try:
                bbox_inter = interp1d(sub_time,
                                      sub_bbox,
                                      kind=interp_kind,
                                      bounds_error=False,
                                      fill_value='extrapolate',
                                      axis=0)
                fill_bbox = bbox_inter(true_time)
            except ValueError as e:
                fill_bbox = np.zeros_like(true_bbox)
            
            # Compare the interpolated points with the reference ones
            mse_err = mse(sref_bbox, fill_bbox)
            
            table.append((interp_kind, frame_rate, mse_err))

interp_df = pd.DataFrame(table, columns=['interpolation_type', 'frame_rate', 'MSE'])

In [ ]:
interp_df.head()